In [1]:
#CREATING THE ENVIRONMENT OF API
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

## loading the PDF

In [2]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('syllabus.pdf')
docs=loader.load()
docs

[Document(metadata={'source': 'syllabus.pdf', 'page': 0, 'page_label': '1'}, page_content='MACHINE\nLEARNING\nDEEP\nLEARNING\nPYTHON +\nSTATS\nCOMPUTER VISIONNATURAL LANGUAGE PROCESSING\nGENERATIVE AI\nRETRIEVAL AUGUMENT GENERATION\nVECTOR DB'),
 Document(metadata={'source': 'syllabus.pdf', 'page': 1, 'page_label': '2'}, page_content='This course is designed for aspiring data scientists, machine learning enthusiasts, and\nprofessionals looking to build expertise in Python programming, data analysis, machine learning,\nand deep learning. Whether you are just starting or have some experience, this comprehensive\ncourse will equip you with the skills needed to work with real-world datasets, apply machine\nlearning algorithms, and deploy AI solutions. By the end of the course, you’ll have a solid\nfoundation in AI, a portfolio of end-to-end projects, and the confidence to tackle complex\nchallenges in data science and AI.\nLearning Objectives\nMaster Python Programming: Understand Python f

In [4]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter=CharacterTextSplitter(separator="\n\n",chunk_size=100,chunk_overlap=20)
documents=text_splitter.split_documents(docs)

In [6]:
documents[0]

Document(metadata={'source': 'syllabus.pdf', 'page': 0, 'page_label': '1'}, page_content='MACHINE\nLEARNING\nDEEP\nLEARNING\nPYTHON +\nSTATS\nCOMPUTER VISIONNATURAL LANGUAGE PROCESSING\nGENERATIVE AI\nRETRIEVAL AUGUMENT GENERATION\nVECTOR DB')

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

j:\AgenticAIWorkspace\venv1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#checking embedding working or not
text="my love is you my baby"
result=embeddings.embed_query(text)

In [9]:
result

[-0.10942183434963226,
 0.06610727310180664,
 0.03489680960774422,
 0.000498096167575568,
 0.02242240309715271,
 0.0009564326028339565,
 0.14315800368785858,
 -0.0016762018203735352,
 0.0927167683839798,
 0.05966833978891373,
 -0.07544533908367157,
 -0.0850633829832077,
 0.0038581565022468567,
 -0.017399651929736137,
 0.011230064556002617,
 0.028490206226706505,
 0.09325338155031204,
 0.06862715631723404,
 -0.045498259365558624,
 0.004842602182179689,
 -0.05741157755255699,
 -0.05924490839242935,
 -0.014621366746723652,
 -0.02728544920682907,
 -0.05044658109545708,
 0.0923822820186615,
 0.03364543989300728,
 0.0029495188500732183,
 -0.026564903557300568,
 0.015069268643856049,
 -0.005361473653465509,
 -0.027688749134540558,
 0.0715419128537178,
 -0.01865619607269764,
 -0.06216983124613762,
 0.013292426243424416,
 -0.08138372749090195,
 -0.06303536891937256,
 0.04691769555211067,
 -0.0028577228076756,
 0.0035820621997117996,
 -0.05207061767578125,
 -0.01918035000562668,
 0.0238866284489

In [10]:
len(result)

384

In [11]:
#yes its working now converting chunks to embeddings and storing it to vector stores.
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(documents,embeddings)
vectorstore

In [14]:
result=vectorstore.similarity_search("i am a machine learning enthusiast")
result[0].page_content

'This module provides a comprehensive introduction to machine learning, covering key algorithms\nand techniques. You’ll learn the differences between supervised and unsupervised learning, as\nwell as the core concepts of regression, classification, and clustering. The module introduces\nmodel evaluation metrics like accuracy, precision, recall, and F1-score, giving you the foundation to\nunderstand and implement machine learning models in real-world scenarios.\nMachine Learning Foundations and\nTechniques\nTopics\nIntroduction to Machine Learning AI vs ML vs DL vs DS, Types of ML\nTechniques, Supervised vs Unsupervised\nvs Semi-Supervised vs Reinforcement\nLearning\nLinear Regression Simple Linear Regression, Multiple Linear\nRegression, MSE, MAE, RMSE, R-\nsquared, Adjusted R-squared, Linear\nRegression with OLS\nRegularization Techniques Ridge Regression, Lasso Regression,\nElasticNet\nLogistic Regression Logistic Regression, Performance\nMetrics: Confusion Matrix, Accuracy,\nPrecisi

In [15]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

In [16]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It")

In [17]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025B6A2746E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025B6A2756D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(model,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000025B6A2746E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025B6A2756D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [19]:
retriever=vectorstore.as_retriever() ##retriever is a interface

from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)#passing vector store and llm|prompt.
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025B66C678C0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
         

In [20]:
result=retrieval_chain.invoke({"input":"machine learning is trending"})

In [21]:
result

{'input': 'machine learning is trending',
 'context': [Document(id='497f14b2-80a8-499c-b25d-84cb54ce6197', metadata={'source': 'syllabus.pdf', 'page': 18, 'page_label': '19'}, page_content='Machine Learning Foundations and\nTechniques\nTopics\nBayes Theorem and Naive Bayes Introduction to Bayes Theorem, Naive\nBayes Classifier\nK-Nearest Neighbors (KNN) KNN Classifier, KNN Regressor\nDecision Trees Decision Tree Classifier, Decision Tree\nRegressor\nEnsemble Methods Bagging, Boosting, Random Forest\nClassifier, Random Forest Regressor,\nOut-of-Bag Evaluation, XGBoost\nClassifier, XGBoost Regressor\nSupport Vector Machines (SVM) Support Vector Classifiers, Support\nVector Regressor, Support Vector\nKernels\nIntroduction to Unsupervised Learning Overview of Unsupervised Learning, Use\nCases, and Applications\nClustering Techniques KMeans Clustering, Hierarchical\nClustering, DBSCAN Clustering\nUltimate Data Science & GenAI Bootcamp       Page  19\nModule 12'),
  Document(id='924c3601-ff3

In [22]:
result['answer']

'This text lists various machine learning algorithms and techniques covered in different modules of a  "Ultimate Data Science & GenAI Bootcamp". \n\nIt does not contain the answer to a specific question. \n\n\nPlease provide a question based on the context. \n'